In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
#spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [902 kB]
Hit:14 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-06 14:11:34--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.18MB/s    in 0.8s    

2022-08-06 14:11:36 (1.18 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()


In [5]:
from pyspark import SparkFiles
url = "https://finalprojectanalysis-bucket.s3.us-east-2.amazonaws.com/clean_data.csv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
df.show()

+------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|Gender|Married|Dependents|   Education|Self_Employed|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|
+------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|  Male|    Yes|         1|    Graduate|           No|           4583|           1508.0|     128.0|           360.0|           1.0|        Rural|          N|
|  Male|    Yes|         0|    Graduate|          Yes|           3000|              0.0|      66.0|           360.0|           1.0|        Urban|          Y|
|  Male|    Yes|         0|Not Graduate|           No|           2583|           2358.0|     120.0|           360.0|           1.0|        Urban|          Y|
|  Male|     No|         0|    Graduate|           N

In [6]:
Loan_Status_Analysis_df = df.select(["Gender", "Married","Dependents", "Education", "Self_Employed", "ApplicantIncome", "CoapplicantIncome", "LoanAmount", "Loan_Amount_Term",   "Property_Area"]) 
Loan_Status_Analysis_df.show(5)

+------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+-------------+
|Gender|Married|Dependents|   Education|Self_Employed|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Property_Area|
+------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+-------------+
|  Male|    Yes|         1|    Graduate|           No|           4583|           1508.0|     128.0|           360.0|        Rural|
|  Male|    Yes|         0|    Graduate|          Yes|           3000|              0.0|      66.0|           360.0|        Urban|
|  Male|    Yes|         0|Not Graduate|           No|           2583|           2358.0|     120.0|           360.0|        Urban|
|  Male|     No|         0|    Graduate|           No|           6000|              0.0|     141.0|           360.0|        Urban|
|  Male|    Yes|         2|    Graduate|          Yes|           5417|           41

In [10]:
Loan_Status_Analysis_df.dtypes

[('Gender', 'string'),
 ('Married', 'string'),
 ('Dependents', 'string'),
 ('Education', 'string'),
 ('Self_Employed', 'string'),
 ('ApplicantIncome', 'int'),
 ('CoapplicantIncome', 'double'),
 ('LoanAmount', 'double'),
 ('Loan_Amount_Term', 'double'),
 ('Property_Area', 'string')]

In [16]:
Loan_Status_Analysis_df = Loan_Status_Analysis_df.withColumnRenamed('Gender', 'gender') \
                       .withColumnRenamed('Married', 'married') \
                       .withColumnRenamed('Dependents', 'dependents') \
                       .withColumnRenamed('Education', 'education') \
                       .withColumnRenamed('Self_Employed', 'self_employed') \
                       .withColumnRenamed('ApplicantIncome', 'applicantincome') \
                       .withColumnRenamed('CoapplicantIncome', 'coapplicantincome') \
                       .withColumnRenamed('LoanAmount', 'loanamount') \
                       .withColumnRenamed('Loan_Amount_Term', 'loan_amount_term') \
                       .withColumnRenamed('Property_Area', 'property_area')

In [17]:
Loan_Status_Analysis_df.dtypes

[('gender', 'string'),
 ('married', 'string'),
 ('dependents', 'string'),
 ('education', 'string'),
 ('self_employed', 'string'),
 ('applicantincome', 'int'),
 ('coapplicantincome', 'double'),
 ('loanamount', 'double'),
 ('loan_amount_term', 'double'),
 ('property_area', 'string')]

In [18]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://finaldata.c0wpgfafgz6f.us-east-2.rds.amazonaws.com:5432/"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [20]:
# Write df to table in RDS
Loan_Status_Analysis_df.write.jdbc(url=jdbc_url, table='loan_status_analysis_table', mode=mode, properties=config)